In [1]:
def crc_(data: list[int]):
    data = bytearray(data)
    crc = 0xFFFF
    for b in data:
        crc ^= b
        for _ in range(0,8):
            bcarry = crc & 0x0001
            crc >>= 1
            if bcarry: 
                crc ^= 0xa001
    rc = hex(crc).replace('0x','').zfill(4)
    crc = [rc[2:4],rc[0:2]]  
    return [int(crc[0],16),int(crc[1],16)]    


In [47]:
import serial
from serial import SerialException
import time

port = "COM5"

def tol_(buf):
    return [hex(i).replace("0x","").zfill(2) for i in list(buf)]

def write_(port, xmd: dict):
    
    rc = { }
    
    device = None
    
    try:
        device = serial.Serial(port, 9600, 8, "N", 1)
        print(f"{port} open '{device.is_open}'")

        if device.is_open:

            for key in xmd.keys():
                
                x = xmd[key]

                device.write(x)
                
                time.sleep(1./9600 * (len(x) * 1000))

                num = device.in_waiting
                buf = device.read(num)
            
                if num >0:

                    rc[key] = {
                        "command": tol_(x),
                        "value": tol_(buf)[len(x):],
                        "num": num
                    }
            

    except SerialException as sx:
        print(sx)

    finally:
        device.close()

    return rc

addr = "50"

TxMap = {
    "unlock": "03 00 69 00 01",
    "wztime": "03 00 6e 00 01",
    "wzstatic": "03 00 6f 00 01",
    "moddelay": "03 00 74 00 01",
    "werror": "03 00 6a 00 01",
    "temperature": "03 00 43 00 01",
    "version": "03 00 2e 00 01",
    "algorithm": "03 00 24 00 01"
}

xmd = {}

for key in TxMap.keys():
    x = [int(addr,16)] + [int(h,16) for h in TxMap[key].split(" ")]
    x = bytes(bytearray([i for i in x + [c for c in crc_(x)]]))
    xmd[key] = x

# rc = write_(port, xmd)

RxMap = {
    "unlockOn": "06 00 69 b5 88",
    "algorithmW": "06 00 24 00 01",
    "unlockOff": "06 00 69 00 00",
    "algorithmR": "03 00 24 00 01"
}

xmd = {}

for key in RxMap.keys():
    x = [int(addr,16)] + [int(h,16) for h in RxMap[key].split(" ")]
    x = bytes(bytearray([i for i in x + [c for c in crc_(x)]]))
    xmd[key] = x

rc = write_(port, xmd)
rc


COM5 open 'True'


{'unlockOn': {'command': ['50', '06', '00', '69', 'b5', '88', '22', 'a1'],
  'value': [],
  'num': 8},
 'algorithmW': {'command': ['50', '06', '00', '24', '00', '01', '05', '80'],
  'value': [],
  'num': 8},
 'unlockOff': {'command': ['50', '06', '00', '69', '00', '00', '54', '57'],
  'value': [],
  'num': 8},
 'algorithmR': {'command': ['50', '03', '00', '24', '00', '01', 'c9', '80'],
  'value': [],
  'num': 8}}